##**Thông tin**


1.   Họ tên: Trần Huỳnh Kỳ Anh
2.   MSSV: 19521216
3.   Ngày thực hiện: 23/11/2021
4.   Nội dung: BT1 + BT2 + BT3



In [2]:
!git clone https://github.com/tranhuynhkyanh/CS116 

Cloning into 'CS116'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 28 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [6]:
import pandas as pd
import numpy as np


In [7]:
dt= pd.read_csv("/content/CS116/Data.csv")

In [8]:
dt

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
678,776715,3,1,1,1,3,2,1,1,1,2
679,841769,2,1,1,1,2,1,1,1,1,2
680,888820,5,10,10,3,7,3,8,10,2,4
681,897471,4,8,6,4,3,4,10,6,1,4


In [9]:
x_data = dt.iloc[:,0:10]
y_data = dt.iloc[:,-1]

In [10]:
x_data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses
0,1000025,5,1,1,1,2,1,3,1,1
1,1002945,5,4,4,5,7,10,3,2,1
2,1015425,3,1,1,1,2,2,3,1,1
3,1016277,6,8,8,1,3,4,3,7,1
4,1017023,4,1,1,3,2,1,3,1,1
...,...,...,...,...,...,...,...,...,...,...
678,776715,3,1,1,1,3,2,1,1,1
679,841769,2,1,1,1,2,1,1,1,1
680,888820,5,10,10,3,7,3,8,10,2
681,897471,4,8,6,4,3,4,10,6,1


In [11]:
y_data

0      2
1      2
2      2
3      2
4      2
      ..
678    2
679    2
680    4
681    4
682    4
Name: Class, Length: 683, dtype: int64

#**Bước 2: Scale các feature**

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
x_data_scaled = scaler.fit_transform(x_data)



#**Bước 3: Chia tập train, test**


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size=0.2)

#**Kfold**

In [14]:
from sklearn.model_selection import KFold
def kFoldSplit(X, y, k):
  kFold = KFold(n_splits=k)
  kFold.get_n_splits(X)

  list_X_train, list_y_train, list_X_test, list_y_test = [], [], [], []

  for i_train, i_test in kFold.split(X):
    list_X_train.append(X[i_train])
    list_y_train.append(y[i_train])
    list_X_test.append(X[i_test])
    list_y_test.append(y[i_test])

  return list_X_train, list_y_train, list_X_test, list_y_test
list_X_train, list_y_train, list_X_test, list_y_test  = kFoldSplit(x_data_scaled, np.array(y_data), 5)
def kfold_rslt(model):
  list_accuracy = []
  sum = 0
  for i in range(len(list_X_train)):
    hist = model.fit(list_X_train[i], list_y_train[i])
    score =hist.score(list_X_test[i], list_y_test[i])
    list_accuracy.append(score)
    sum += score
  return list_accuracy,sum/5


#**BT 1: GBM to train**

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)
acc_list,average = kfold_rslt(clf)

In [16]:
print("Độ chính xác từng fold:", acc_list)
print("Độ chính xác trung bình:", average)

Độ chính xác từng fold: [0.8905109489051095, 0.948905109489051, 0.948905109489051, 0.9852941176470589, 0.9632352941176471]
Độ chính xác trung bình: 0.9473701159295835


#**BT 2: XGBoost**

In [17]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=x_data_scaled,label=np.array(y_data))

In [18]:
xg_clf = xgb.XGBClassifier()
acc_list_1,average_1 = kfold_rslt(xg_clf)

In [19]:
print("Độ chính xác từng fold:", acc_list_1)
print("Độ chính xác trung bình:", average_1)

Độ chính xác từng fold: [0.9197080291970803, 0.948905109489051, 0.9635036496350365, 0.9852941176470589, 0.9705882352941176]
Độ chính xác trung bình: 0.9575998282524688


#So sánh kết quả BT1 và BT2

In [20]:
import pandas as pd
df = pd.DataFrame([acc_list,acc_list_1])
df.rename(index={0:"GBM",1:"XGB"}, inplace=True)

In [21]:
df

,0,1,2,3,4
GBM,0.890511,0.948905,0.948905,0.985294,0.963235
XGB,0.919708,0.948905,0.963504,0.985294,0.970588


#**BT3: loan_data.csv**

In [22]:
dt_1 = pd.read_csv("/content/CS116/loan_data.csv")

In [23]:
dt_1

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


##Cleaning and train test split

In [48]:
dt_1 = dt_1[dt_1['LoanAmount'].notnull()]

In [65]:
x_data = dt_1.iloc[:,1:12]
y_data = dt_1.iloc[:,-1]
x_data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban


In [66]:
#Encode
x_data['Married'] = x_data['Married'].astype('category')
x_data['Gender'] = x_data['Gender'].astype('category')
x_data['Education'] = x_data['Education'].astype('category')
x_data['Self_Employed'] = x_data['Self_Employed'].astype('category')
x_data['Property_Area'] = x_data['Property_Area'].astype('category')
x_data = pd.get_dummies(x_data)

y_data = y_data.astype('category')
y_data = pd.get_dummies(y_data)

In [67]:
y_data

,N,Y
1,1,0
2,0,1
3,0,1
4,0,1
5,0,1
...,...,...
609,0,1
610,0,1
611,0,1
612,0,1


In [69]:
x_data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0
610,4106,0.0,40.0,180.0,1.0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0
611,8072,240.0,253.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
612,7583,0.0,187.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1


In [70]:
scaler = StandardScaler() 
x_data_scaled = scaler.fit_transform(x_data)


In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size=0.2)

##GBM

In [72]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)
acc_list,average = kfold_rslt(clf)

In [73]:
print("Độ chính xác từng fold:", acc_list)
print("Độ chính xác trung bình:", average)

Độ chính xác từng fold: [0.8905109489051095, 0.948905109489051, 0.948905109489051, 0.9852941176470589, 0.9632352941176471]
Độ chính xác trung bình: 0.9473701159295835


##XGB

In [74]:
data_dmatrix = xgb.DMatrix(data=x_data_scaled,label=np.array(y_data))
xg_clf = xgb.XGBClassifier()
acc_list_1,average_1 = kfold_rslt(xg_clf)

In [75]:
print("Độ chính xác từng fold:", acc_list_1)
print("Độ chính xác trung bình:", average_1)

Độ chính xác từng fold: [0.9197080291970803, 0.948905109489051, 0.9635036496350365, 0.9852941176470589, 0.9705882352941176]
Độ chính xác trung bình: 0.9575998282524688
